# Construyendo un Motor de Búsqueda desde 0

Pasos:
1. Configurar el ambiente para el desarrollo.
   - Configurar ambiente virtual Python.
   - Configurar Elasticsearch instance.
2. Normalizar contenido para búsqueda. Usaremos una versión reducida de este conjunto de datos disponible públicamente: https://www.kaggle.com/dhruvildave/wikibooks-dataset
3. Crear indice Elasticsearch con contenido normalizado.
4. Construir sentencias de búsqueda para Elasticsearch.
5. Medir el desempeño de mi motor con cada sentencia de búsqueda y escoger la mejor.
6. Crear UI y ensamblar todos lo componentes.
7. BONUS: Integrar modelos de lenguaje para:
    - Mejorar reranking
    - Extraer y mostrar respuesta al usuario contenida en los parrafos

### 1. Configurar el ambiente para el desarrollo

In [ ]:
# instalar dependencias requeridas
!pip install -r requirements.txt

In [ ]:
# iniciar elasticsearch instance
!docker-compose up

### 2. Normalizar contenido para búsqueda

In [ ]:
# lectura del dataset archivo zip e imprimir numero total de registros

#### 2.1 Reconocimiento del formato

In [ ]:
# convertir dataset a listado de dicts e imprimir alguno de los elementos

In [ ]:
# from IPython.core.display import display, HTML

#### 2.2 Conversión HTML a formato intermedio Markdown

In [ ]:
# HTML de referencia
html = "<b>Misión TIC 2022</b>" + \
"<br /><br /><br />" + \
"Rutas" + \
"<ul>" + \
"<li><em>Ruta</em> <b>1</b></li>" + \
"<li><a href='#example'>Ruta 2</a></li>" + \
"</ul>" + \
"<a href='https://www.misiontic2022.gov.co/portal/Secciones/Inscripciones-2021/'>Requisitos Ruta 2</a>:" + \
"<p>Si eres mayor de 15        años, quieres aprender a programar y tienes la disponibilidad de 30 horas semanales para realizar la formación. Deberás registrate en la web, seleccionar la ruta 2 y diligenciar el formulario de inscripción.</p>" + \
"Universidades:" + \
"<br />" + \
"* UniNorte<br />· UTP<br />- Universidad Nacional<br />La universidad nacional ...<br />- UNAB" + \
"<br /><br /><br />" + \
"<span>[</span><a href='#example' title='Editar sección'>editar</a><span>]</span>Tiempos:" + \
"<br />" + \
"<table><tr><th>Ruta</th><th>Tiempo</th><th>Edad</th></tr><tr><td>Ruta 1</td><td>11 horas</td><td>12</td></tr><tr><td>Ruta 2</td><td>30 horas</td><td>18</td></tr></table>" + \
"<img src='https://misiontic2022.mintic.gov.co/mtv2/assets/assets/images/logo-mision.png' alt='mintic 2022' width='500'>"
# imprimir

In [ ]:
# probar conversor inicial: html2text import HTML2Text

#### 2.3 Ajustar conversor para lectura óptima

In [ ]:
# ignorar enlaces, ignorar enfasis, ignorar imagenes y no ajustar ancho de texto

#### 2.4 Normalizar listas

In [ ]:
# reescribir los elementos de lista usando un correcto formato de Markdown (eg.  * list item) - re.sub

#### 2.5 Normalizar tablas

In [ ]:
# 1. "| " -> " | "
# 2. eliminar "-----|----|---"
# 3. "    | " -> " | "

#### 2.6 De vuelta a HTML simplificado (MD -> HTML)

In [ ]:
# from mistletoe import markdown

#### 2.7 Conversión HTML a Text

In [ ]:
# from bs4 import BeautifulSoup

#### 2.8 Manejo extra espacios

In [ ]:
# 1. eliminar espacios a los extremos
# 2. reemplazar tabulaciones y espacios dobles por uno dentro del texto
# 3. reemplazar salto de linea dobles por uno dentro del texto

#### 2.9 Borrar artefactos innecesarios

In [ ]:
# borrar enlaces [editar]

#### 2.10 Divide el texto en párrafos

#### 2.11 Filtrar párrafos - definir datos en alcance

In [ ]:
# Alcance definido:
# 1. solo incluye los párrafos con más de 50 letras

#### 2.12 Crear función global de normalización de contenido (unir piezas)

In [ ]:
def normalise_content(input_html):
    # implement
    return None

#### 2.13 Crear función de normalización de títulos

In [ ]:
# Wikilibros: Wikichicos/La Tierra/Los continentes/Europa/Clima -> La Tierra, Los continentes, Europa, Clima
import re
def normalise_title(input_title):
    # implement
    return None

#### 2.14 Crear elementos que indexaremos a Elasticsearch - formato definido:
{
    "bookId": 0, 
    "bookTitle": "", 
    "bookURL": "", 
    "paragraphId": 0,
    "paragraphText": ""
}

In [ ]:
# from tqdm import tqdm

In [ ]:
# imprimir total numero de parrafos a almacenar en ES

In [ ]:
# persistimos contenido normalizado en forma de parrafos al disco duro en formato JSON en dataset/processed/ (crear directorio si no existe)

### 3. Crear indice en Elasticsearch con contenido normalizado

##### 3.1. Instanciamos cliente para poder interactuar con Elasticsearch

##### 3.2. Función para crear índice (indexar contenido) en Elasticsearch

In [ ]:
import os
from elasticsearch import helpers

def create_index(index_action_list, index_name):
    # si el index existe, borrarlo (es.indices.exists y es.indices.delete)
    # crear request body con mis mappings 
    # crear index (es.indices.create)
    # crear elementos en paralelo usando helper.parallel_bulk
    pass

##### 3.4. Creación del índice

In [ ]:
# lectura de archivo con parrafos procesados
import json
with open("dataset/processed/paragraphs.json") as fp:
    paragraphs = json.load(fp)

In [ ]:
# creación de indice con contenido normalizado generado en el paso 2
index_name = "wikibooks-search-index"

### 4. Construir sentencias de búsqueda (query) para Elasticsearch

#### 4.1. Funciones para ejecutar búsqueda de contenido en elasticsearch 
Crear formato a partir de la respuesta de ES y convertir a DF:
{
    "bookId": 0, 
    "bookTitle": "", 
    "bookURL": "", 
    "paragraphId": 0,
    "paragraphText": "",
    "esScore": 0
}

In [ ]:
# from elasticsearch import helpers
# from itertools import islice
def find_by_query(query, index_name="wikibooks-search-index", size=100):
    # consulta a Elasticsearch helper.scan conservando el orden

In [ ]:
# opción para NO truncar texto cuando es muy largo
# pd.set_option("display.max_colwidth", None)

#### 4.2 Demo diferentes tipos de queries soportados por Elasticsearch

###### 4.2.1 Query usando un simple campo de búsqueda - paragraphText

In [ ]:
def es_query_1(user_query):
    es_query = {
        "query": {
            "bool": {
                "must": {"match": {"paragraphText": user_query}}
            }
        }
    }
    return es_query

###### 4.2.2 Query usando múltiples campos de búsqueda - paragraphText y bookTitle

In [ ]:
def es_query_2(user_query):
    es_query = {
        "query": {
            "multi_match": {
                "query": user_query,
                "fields": ["paragraphText", "bookTitle"],
                "tie_breaker": 1.0
            }
        }
    }
    return es_query

###### 4.2.3 Query impulsando campos de búsqueda paragraphText y bookTitle

In [ ]:
def es_query_3(user_query):
    es_query = {
        "query": {
            "multi_match": {
                "query": user_query,
                "fields": ["paragraphText", "bookTitle^2.0"],
                "tie_breaker": 1.0
            }
        }
    }
    return es_query

###### 4.2.4 Query usando múltiples campos de búsqueda con soporte a coincidencia exacta de frases

In [ ]:
import re
def es_query_4(user_query):
    
    match_queries = [
        {"match": {"paragraphText": user_query}},
        {"match": {"bookTitle": user_query}}
    ]
    
    # identificar frases en mi user_query
    phrases = re.findall(r'"([^"]+)"', user_query)
    # agregar frases a mi ES query
    for phrase in phrases:
        match_queries.append({"match": {"paragraphText": phrase}})
        match_queries.append({"match": {"bookTitle": phrase}})
    
    es_query = {
        "query": {
            "bool": {
                "should": match_queries
            }
        }
    }
    return es_query

### 5. Midiendo el desempeño de mi motor de búsqueda

##### 5.1 datos preparados para medición de desempeño

In [ ]:
query_relevant_answer_pairs = [
    {
        "text": "extensión total de la Antártida",
        "relevant_results": {
            "2803-1": "Su extensión total es de aproximadamente 14,2 millones de km2 en verano. Durante el invierno, la Antártida dobla su tamaño a causa de la gran cantidad de hielo marino que se forma en su periferia. El verdadero límite de la Antártida no es el litoral del continente en sí mismo, sino la Convergencia Antártica , que es una zona claramente definida en el extremo sur de los océanos Atlántico, Índico y Pacífico, entre los 48° y los 60° latitud S. En este punto, las corrientes frías que fluyen hacia el Norte desde la Antártida se mezclan con corrientes más cálidas en dirección Sur. La Convergencia Antártica marca una clara diferencia física en los océanos. Por estas razones el agua que rodea al continente antártico se considera un océano en sí mismo, a menudo llamado océano Glacial Antártico o Meridional."
        }
    },
    {
        "text": "dorsales submarinas",
        "relevant_results": {
            "3288-15": "Las dorsales son cordilleras submarinas formadas al ponerse en contacto el magma del interior de la Tierra con las aguas de los océanos. Por el centro del Atlántico corre una dorsal con forma de S, de unos 15 000 km de longitud, casi paralela a los continentes que da lugar a las islas Azores. El océano Índico también está recorrido por una dorsal de Norte a Sur y en el Pacífico también existen dorsales en dirección Noreste-Sureste; las islas Hawai están situadas sobre una de estas dorsales ."
        }
    },
    {
        "text": "principales gases en la atmósfera terrestre",
        "relevant_results": {
            "527-12": "La atmósfera terrestre es la parte gaseosa de la Tierra, siendo por esto la capa más externa. Está constituida por varios gases que varían en cantidad. Esta mezcla de gases que forma la atmósfera recibe genéricamente el nombre de aire. Los principales elementos que la componen son el oxígeno (21%) y el nitrógeno (78%)."
        }
    },
    {
        "text": "anatomía de los mayas",
        "relevant_results": {
            "318-10": "Eran de baja estatura, hombros anchos, pecho robusto, piernas cortas y musculosas, el rostro alargado y los pómulos salientes. Su estatura era, en término medio, de 1,65 m los varones y 1,42 m las mujeres. El cabello negro y lacio y la piel de color cobrizo confirma su procedencia del norte de Asia."
        }
    },
    {
        "text": "potencia máxima rueda hidráulica",
        "relevant_results": {
            "1965-3": "Los antiguos aprovechaban ya la energía del agua; utilizaban ruedas hidráulicas para moler trigo. Durante la Edad Media , las enormes ruedas hidráulicas de madera desarrollaban una potencia máxima de cincuenta caballos."
        }
    },
    {
        "text": "¿Cuándo se firmó el tratado antártico?",
        "relevant_results": {
            "2803-5": "Siete países ( Argentina, Australia, Chile, Francia, Gran Bretaña, Nueva Zelanda y Noruega ) reivindican la soberanía de ciertos territorios de la Antártida, pero desde el Tratado Antártico de 1961 estas demandas han sido abandonadas en favor de la cooperación internacional en las investigaciones científicas."
        }
    },
    {
        "text": "¿Cuánta sal hay en promedio en los océanos?",
        "relevant_results": {
            "3288-4": "El agua de los océanos contiene una media de 36 gramos por litro de sales. Esto se expresa diciendo que la salinidad media de los océanos es del 36 por 1 000. Entre las sales disueltas, el cloruro de sodio (sal común) es la más abundante. En las costas de las lugares donde hace mucho calor y la evaporación es grande el hombre extrae la sal de las salinas, por ejemplo en el Mediterráneo. también tiene disueltas pequeñas cantidades de yodo, fósforo y cobre."
        }
    },
    {
        "text": "¿Cuántos años tiene la tierra?",
        "relevant_results": {
            "2375-0": "Hace alrededor de 4550 millones de años atrás se formaron la Tierra y los otros planetas del Sistema Solar a partir de la nebulosa solar; una masa en forma de disco compuesta del polvo y gas que aún quedaba de la formación del Sol. Este proceso de formación de la Tierra tuvo lugar en un plazo de 10-20 millones de años."
        }
    },
    {
        "text": "¿Dónde vivía la abuelita de Caperucita?",
        "relevant_results": {
            "3152-2": "Caperucita tenía una abuela que vivía en una casita al otro lado del bosque, por lo que para ir a verla tenía que cruzar todo el bosque. En el bosque,vivían animales que no eran peligrosos: como los ciervos, los conejos y muchas especies de pájaros, pero también vivía en ese bosque un animal que sí podía ser peligroso, sobre todo para los niños: era un lobo que cuando tenía mucha hambre atacaba a las personas."
        }
    },
    {
        "text": "¿De qué estaba hecha la casa de Hansel y Gretel?",
        "relevant_results": {
            "2997-2": "Llegaron a una casita hecha de pan de jengibre, pastel y azúcar moreno,",
            "2997-4": "Después de dos días perdidos en el bosque, cuando ya no sabían más que hacer, los niños se detienen a escuchar el canto de un pájaro blanco al cual luego siguen hasta llegar a una casita hecha de pan de jengibre, pastel y azúcar morena. Hansel y Gretel empezaron a comer, pero lo que no sabían era que esta casita era la trampa de una vieja bruja para encerrarlos y luego comérselos."
        }
    }
]

##### 5.2 Funciones para medición de desempeño

In [ ]:
def recall_at_k(relevant_results, candidates, k):
    candidates_in_k = candidates[:min(k, len(candidates))]
    tp = len(set(relevant_results).intersection(set(candidates_in_k)))
    fn = len(set(relevant_results) - set(candidates_in_k))
    return tp / min(k, (tp + fn)) # same as tp / min(k, len(relevant_results))

def precision_at_k(relevant_results, candidates, k):
    candidates_in_k = candidates[:min(k, len(candidates))]
    tp = len(set(relevant_results).intersection(set(candidates_in_k)))
    fp = len(set(candidates_in_k) - set(relevant_results))
    return tp / min(k, (tp + fp)) # same as tp / min(k, len(candidates_in_k))

def average_precision_at_k(relevant_results, candidates, k):
    precision_acum = 0
    for i in range(k):
        if candidates[i] in relevant_results:
            rank = i + 1
            precision_acum += precision_at_k(relevant_results, candidates, rank)
    return precision_acum / min(k, len(relevant_results))

def compute_engine_performance(queries_with_relevant_answers, k_list, query_builder_functions):
    """
    Iterar sobre:
    - todos los tipos de query de Elasticsearch
    - k's = numbero de resultados tope que vamos a cubrir (top 10, 20, etc)
    - todos mis resultados relevantes previamente anotados
    
    Crear un DataFrame con la siguiente información:
    - tipo de query
    - K
    - recall
    - average_precision
    """
    return None

##### 5.3 Medición de desempeño de diferentes Elasticsearch queries

In [ ]:
performance_summary_es = compute_engine_performance(query_relevant_answer_pairs, [10, 20], [es_query_1, es_query_2, es_query_3, es_query_4])

### 6 Creamos nuestra función de búsqueda final
Formato de respuesta esperado:
{
    "bookId": 0, 
    "bookTitle": "", 
    "bookURL": "", 
    "paragraphId": 0,
    "paragraphText": "",
    "esScore": 0,
    "questionText": ""
}

In [ ]:
def format_es_response(user_question, es_candidates):
    results = list()
    # implementar
    return results

def search_candidates(user_question, index_name="wikibooks-search-index", size=20, es=Elasticsearch()):
    # implementar despues de decidir qué tipo de query tomar
    return format_es_response(user_question, results)
    

### 7 BONUS: Integrar modelos de reordenamiento

##### 7.1 Funciones para llamado de modelos de reordenamiento (reranking)

##### 7.2 Funciones para medición de desempeño con reordenamiento

##### 7.3 Medición de desempeño de la función de reordenamiento (reranking)